In [ ]:
%install '.package(path: "$cwd/FastaiNotebook_07_batchnorm")' FastaiNotebook_07_batchnorm
%install '.package(path: "$cwd/FastaiNotebook_08a_heterogeneous_dictionary")' FastaiNotebook_08a_heterogeneous_dictionary

## Load data

In [ ]:
import FastaiNotebook_07_batchnorm
import FastaiNotebook_08a_heterogeneous_dictionary
//%include "EnableIPythonDisplay.swift"
//IPythonDisplay.shell.enable_matplotlib("inline")

In [ ]:
// export
import Path
import TensorFlow

In [ ]:
let data = mnistDataBunch(flat: true)

In [ ]:
let (n,m) = (60000,784)
let c = 10
let nHid = 50

In [ ]:
func modelInit() -> BasicModel {return BasicModel(nIn: m, nHid: nHid, nOut: c)}

## Stateful optimizer

In [ ]:
//export
open class StatDelegate<Scalar: TensorFlowFloatingPoint> {
    open var name: String { return "" }
    var defaultConfig: HeterogeneousDictionary { return HeterogeneousDictionary() }
    func update(
        state: inout [String: Tensor<Scalar>],
        for param: Tensor<Scalar>,
        along direction: Tensor<Scalar>,
        config: inout HeterogeneousDictionary
    ) { }
}

//export
open class StepDelegate<Scalar: TensorFlowFloatingPoint> {
    var defaultConfig: HeterogeneousDictionary { return HeterogeneousDictionary() }
    func update(
        param: inout Tensor<Scalar>,
        along direction: inout Tensor<Scalar>,
        state: [String: Tensor<Scalar>],
        config: inout HeterogeneousDictionary
    ) { }
}

In [ ]:
//export
class StatefulOptimizer<Model: Layer,
                        Scalar: TensorFlowFloatingPoint>: Optimizer
    where Model.AllDifferentiableVariables == Model.CotangentVector{
    var config: HeterogeneousDictionary
    var learningRate: Float {
        get { return config[LearningRate()]!} 
        set { config[LearningRate()] = newValue }
    }
    var states: [String: Model.AllDifferentiableVariables]
    var statDelegates: [StatDelegate<Scalar>]
    var stepDelegates: [StepDelegate<Scalar>]
    init(
        stepDelegates: [StepDelegate<Scalar>],
        statDelegates: [StatDelegate<Scalar>],
        config: HeterogeneousDictionary
    ) {
        self.config = HeterogeneousDictionary()
        states = [:]
        for stepDelegate in stepDelegates {
            self.config.merge(stepDelegate.defaultConfig) { (_, new) in new }
        }
        for statDelegate in statDelegates {
            self.config.merge(statDelegate.defaultConfig) { (_, new) in new }
            states[statDelegate.name] = Model.AllDifferentiableVariables.zero
        }
        self.config.merge(config) { (_, new) in new }
        self.stepDelegates = stepDelegates
        self.statDelegates = statDelegates
    }
    func update(
        _ model: inout Model.AllDifferentiableVariables,
        along direction: Model.CotangentVector
    ) {
        for kp in model.recursivelyAllWritableKeyPaths(to: Tensor<Scalar>.self) {
            var grad = direction[keyPath: kp]
            var state = states.mapValues(){$0[keyPath: kp]}
            for statDelegate in statDelegates {
                statDelegate.update(
                    state: &state,
                    for: model[keyPath: kp],
                    along: grad,
                    config: &config
                )
            }
            for n in states.keys { states[n]![keyPath: kp] = state[n]! }
            for stepDelegate in stepDelegates {
                stepDelegate.update(
                    param: &model[keyPath: kp],
                    along: &grad,
                    state: state,
                    config: &config
                )
            }
        }
    }
}

In [ ]:
//export
class SGDStep: StepDelegate<Float> {
    override func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) {
        param -= direction * config[LearningRate()]!
    }
}

In [ ]:
//export
public struct WeightDecayKey: HetDictKey, Equatable {
    public static var key = WeightDecayKey() as AnyHashable
    public typealias ValueType = Float
}

class WeightDecay: StepDelegate<Float> {
    override var defaultConfig: HeterogeneousDictionary { return HeterogeneousDictionary([WeightDecayKey(): 0.0]) }
    override func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) {
        param *= 1 - config[LearningRate()]! * config[WeightDecayKey()]!
    }
}

In [ ]:
//export

class L2Regularization: StepDelegate<Float> {
    override var defaultConfig: HeterogeneousDictionary { return HeterogeneousDictionary([WeightDecayKey(): 0.0]) }
    override func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) {
        direction += config[WeightDecayKey()]! * param
    }
}

In [ ]:
//export

public struct Momentum: HetDictKey, Equatable {
    public static var key = Momentum() as AnyHashable
    public typealias ValueType = Float
}

public struct MomentumDampening: HetDictKey, Equatable {
    public static var key = MomentumDampening() as AnyHashable
    public typealias ValueType = Float
}

class AverageGrad: StatDelegate<Float> {
    override var defaultConfig: HeterogeneousDictionary { return HeterogeneousDictionary([Momentum(): Float(0.9)]) }
    let dampened: Bool
    init(dampened: Bool = false) { self.dampened = dampened }
    override var name: String { return "averageGrad" }
    override func update(
        state: inout [String: Tensor<Float>],
        for param: Tensor<Float>,
        along direction: Tensor<Float>,
        config: inout HeterogeneousDictionary
    ) {
        state["averageGrad"]! *= config[Momentum()]!
        config[MomentumDampening()] = 1.0 - (dampened ? config[Momentum()]! : 0.0)
        state["averageGrad"]! += config[MomentumDampening()]! * direction
    }
}

In [ ]:
let opt = StatefulOptimizer<BasicModel, Float>(stepDelegates: [SGDStep()], statDelegates: [], 
                                               config: HeterogeneousDictionary([LearningRate(): Float(0.01)]))

In [ ]:
let learner = Learner(data: data, lossFunction: softmaxCrossEntropy, optimizer: opt, initializingWith: modelInit)
let recorder = learner.makeDefaultDelegates(metrics: [accuracy])
learner.delegates.append(learner.makeNormalize(mean: mnistStats.mean, std: mnistStats.std))

In [ ]:
learner.fit(2)

In [ ]:
//export
class MomentumStep: StepDelegate<Float> {
    override func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) {
        param -= config[LearningRate()]! * state["averageGrad"]!
    }
}

In [ ]:
let opt = StatefulOptimizer<BasicModel, Float>(stepDelegates: [MomentumStep()], statDelegates: [AverageGrad()], 
                                               config: HeterogeneousDictionary([LearningRate(): Float(0.01)]))

In [ ]:
let learner = Learner(data: data, lossFunction: softmaxCrossEntropy, optimizer: opt, initializingWith: modelInit)
let recorder = learner.makeDefaultDelegates(metrics: [accuracy])
learner.delegates.append(learner.makeNormalize(mean: mnistStats.mean, std: mnistStats.std))

In [ ]:
learner.fit(2)

In [ ]:
//export

public struct SquareMomentum: HetDictKey, Equatable {
    public static var key = SquareMomentum() as AnyHashable
    public typealias ValueType = Float
}

public struct SquareMomentumDampening: HetDictKey, Equatable {
    public static var key = SquareMomentumDampening() as AnyHashable
    public typealias ValueType = Float
}


class AverageSquaredGrad: StatDelegate<Float> {
    override var defaultConfig: HeterogeneousDictionary { return HeterogeneousDictionary([SquareMomentum(): Float(0.99)]) }
    let dampened: Bool
    init(dampened: Bool = false) { self.dampened = dampened }
    override var name: String { return "averageSquaredGrad" }
    override func update(
        state: inout [String: Tensor<Float>],
        for param: Tensor<Float>,
        along direction: Tensor<Float>,
        config: inout HeterogeneousDictionary
    ) {
        state["averageSquaredGrad"]! *= config[SquareMomentum()]!
        config[SquareMomentumDampening()] = 1.0 - (dampened ? config[SquareMomentum()]! : 0.0)
        state["averageSquaredGrad"]! += config[SquareMomentumDampening()]! * direction.squared()
    }
}

In [ ]:
//export
class StepCount: StatDelegate<Float> {
    override var name: String { return "step" }
    override func update(
        state: inout [String: Tensor<Float>],
        for param: Tensor<Float>,
        along direction: Tensor<Float>,
        config: inout HeterogeneousDictionary
    ) {
        state["step"]! += 1.0
    }
}

In [ ]:
//export
func debias<Scalar: TensorFlowFloatingPoint>(
    momentum: Scalar,
    dampening: Scalar,
    step: Tensor<Scalar> 
) -> Tensor<Scalar> {
    return dampening * (1 - pow(momentum, step)) / (1 - momentum)
}

In [ ]:
//export

public struct Epsilon: HetDictKey, Equatable {
    public static var key = Epsilon() as AnyHashable
    public typealias ValueType = Float
}


/*

class MomentumStep: StepDelegate<Float> {
    override func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) {
        param -= config[LearningRate()]! * state["averageGrad"]!
    }
}

*/

class AdamStep: StepDelegate<Float> {
    override var defaultConfig: HeterogeneousDictionary { return HeterogeneousDictionary([Epsilon(): Float(1e-5)]) }
    override func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) {
        let debiasedLearningRate = config[LearningRate()]! / debias(
            momentum: config[Momentum()]!,
            dampening: config[MomentumDampening()]!,
            step: state["step"]!
        )
        let debiasedRMSGrad = sqrt(state["averageSquaredGrad"]! / debias(
            momentum: config[SquareMomentum()]!,
            dampening: config[SquareMomentumDampening()]!,
            step: state["step"]!
        )) + config[Epsilon()]!
        param -= debiasedLearningRate * state["averageGrad"]! / debiasedRMSGrad
    }
}

In [ ]:
let opt = StatefulOptimizer<BasicModel, Float>(
    stepDelegates: [AdamStep()], 
    statDelegates: [AverageGrad(), AverageSquaredGrad(), StepCount()], 
    config: HeterogeneousDictionary([LearningRate(): Float(0.01)]))

In [ ]:
let learner = Learner(data: data, lossFunction: softmaxCrossEntropy, optimizer: opt, initializingWith: modelInit)
let recorder = learner.makeDefaultDelegates(metrics: [accuracy])
learner.delegates.append(learner.makeNormalize(mean: mnistStats.mean, std: mnistStats.std))

In [ ]:
learner.fit(2)

In [ ]:
class LambStep: StepDelegate<Float> {
    override var defaultConfig: HeterogeneousDictionary {
        return HeterogeneousDictionary([Epsilon(): Float(1e-6), WeightDecayKey(): Float(0.0)])
    }
    override func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) {
        let debiasedAverageGrad = state["averageGrad"]! / debias(
            momentum: config[Momentum()]!,
            dampening: config[MomentumDampening()]!,
            step: state["step"]!
        )
        let debiasedRMSGrad = sqrt(state["averageSquaredGrad"]! / debias(
            momentum: config[SquareMomentum()]!,
            dampening: config[SquareMomentumDampening()]!,
            step: state["step"]!
        ) + config[Epsilon()]!)
        let step = debiasedAverageGrad / debiasedRMSGrad + config[WeightDecayKey()]! * param
        let r1 = sqrt((param * param).mean())
        let r2 = sqrt((step * step).mean())
        let factor = min(r1 / r2, Float(10.0))
        param -= config[LearningRate()]! * factor * step
    }
}

## Export

In [ ]:
notebookToScript(fname: (Path.cwd / "09_optimizer.ipynb").string)